In [42]:
#1. Data Collection
import pandas as pd
dataset1 = pd.read_csv("preprocessed_LoadBalancer.csv", index_col= None)

In [43]:
dataset1

,Server Status,Data Progress,Network Traffic (MB/s),Request Size (MB),Threshold,Server Load,Response Time (ms),Priority (0/1)
0,s2,inprogress,86.55,44.09,0.77,medium,359,1
1,s3,inprogress,37.39,346.07,0.95,low,200,0
2,s6,complete,33.81,109.68,0.78,low,447,1
3,s5,inprogress,45.27,283.95,0.92,low,408,1
4,s4,complete,74.16,99.97,0.91,medium,320,1
...,...,...,...,...,...,...,...,...
1249,s7,inprogress,300.00,128.00,8.00,medium,172,1
1250,s4,complete,569.00,411.00,3.00,high,857,0
1251,s7,inprogress,703.00,79.00,3.00,high,635,1
1252,s4,inprogress,147.00,562.00,9.00,high,823,0


In [78]:
# 4) Feature Selection and Model Creation
# convert the categorial columns to numerical using one hot encoding get_dummies
df2 = dataset1
df2=pd.get_dummies(df2, drop_first=True)
indep_X = df2.drop('Priority', 1)
dep_Y = df2['Priority']

In [87]:
# 4 A 1) Feature Selection: ChiSquare Chi2  -- K 7 returns the highest chi2 score with 56%
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2 
def selectKbest(indep_X, dep_Y, n ):
    test = SelectKBest(score_func=chi2, k= n)
    fit1 = test.fit(indep_X, dep_Y)
    selectk_feature = fit1.transform(indep_X)
    print("selectkbest k:", n)
    return selectk_feature
def split_scalar(indep_X, dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size =0.25, random_state= 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.fit_transform(X_test)
        return X_train, X_test, y_train, y_test
def confusionMatrix_prediction(classifier, X_test):
    y_pred = classifier.predict(X_test)
    
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    
    Accuracy = accuracy_score(y_test, y_pred)
    
    report = classification_report(y_test, y_pred)
    return classifier, Accuracy, report, X_test, y_test, cm
def logistic(X_train, y_train, X_test):
    from sklearn.linear_model import LogisticRegression
    classifier = LogisticRegression(random_state=0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = confusionMatrix_prediction(classifier, X_test)
    return classifier, Accuracy, report, X_test, y_test, cm
def svm_linear(X_train, y_train, X_test):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'linear', random_state=0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = confusionMatrix_prediction(classifier, X_test)
    return classifier, Accuracy, report, X_test, y_test, cm
def svm_NL(X_train, y_train, X_test):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'rbf', random_state=0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = confusionMatrix_prediction(classifier, X_test)
    return classifier, Accuracy, report, X_test, y_test, cm
def Navie(X_train, y_train, X_test):
    from sklearn.naive_bayes import GaussianNB
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = confusionMatrix_prediction(classifier, X_test)
    return classifier, Accuracy, report, X_test, y_test, cm
def knn(X_train, y_train, X_test):
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors=5, metric = 'minkowski', p = 2)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = confusionMatrix_prediction(classifier, X_test)
    return classifier, Accuracy, report, X_test, y_test, cm
def Decision(X_train, y_train, X_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state=0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = confusionMatrix_prediction(classifier, X_test)
    return classifier, Accuracy, report, X_test, y_test, cm
def random(X_train, y_train, X_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    classifier, Accuracy, report, X_test, y_test, cm = confusionMatrix_prediction(classifier, X_test)
    return classifier, Accuracy, report, X_test, y_test, cm

In [103]:
def selectK_Classification_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    dataframe = pd.DataFrame(index=['ChiSquare'], columns = ['Logistic', 'SVMl','SVMnl', 'KNN', 'Navie', 'Decision', 'Random' ])
    print(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    for number, idex in enumerate(dataframe.index):
        dataframe['Logistic'][idex]=acclog[number]
        dataframe['SVMl'][idex]=accsvml[number]
        dataframe['SVMnl'][idex]=accsvmnl[number]
        dataframe['KNN'][idex]=accknn[number]
        dataframe['Navie'][idex]=accnav[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe

kbest = selectKbest(indep_X, dep_Y, 2)
acclog = [] 
accsvml= [] 
accsvmnl= []
accknn= [] 
accnav= [] 
accdes= []
accrf = []

X_train, X_test, y_train, y_test = split_scalar (kbest, dep_Y)
classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train, y_train, X_test)
acclog.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train, y_train, X_test)
accsvml.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_NL(X_train, y_train, X_test)
accsvmnl.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = knn(X_train, y_train, X_test)
accknn.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train, y_train, X_test)
accnav.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train, y_train, X_test)
accdes.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = random(X_train, y_train, X_test)
accrf.append(Accuracy)


selectkbest k: 2


C:\Users\mpriy\anaconda3\envs\aimlVirtualEnvironment\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\mpriy\anaconda3\envs\aimlVirtualEnvironment\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Users\mpriy\anaconda3\envs\aimlVirtualEnvironment\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Users\mpriy\anaconda3\envs\aimlVirtualEnvironment\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


In [104]:
result = selectK_Classification_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)

1 1 1 1 1 1 1


In [90]:
result   
#7
# Random 56% is the highest chi2 score

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.512739,0.519108,0.519108,0.490446,0.525478,0.531847,0.563694


In [93]:
result
#9

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.512739,0.522293,0.55414,0.544586,0.531847,0.531847,0.528662


In [96]:
result
#5

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.531847,0.531847,0.538217,0.493631,0.531847,0.506369,0.55414


In [99]:
result
#4

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.535032,0.531847,0.541401,0.509554,0.525478,0.493631,0.506369


In [102]:
result
#3

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.496815,0.496815,0.471338,0.493631,0.484076,0.496815,0.544586


In [105]:
result
#2

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.525478,0.525478,0.509554,0.509554,0.515924,0.496815,0.525478


In [110]:
# 4 A 2) Feature Selection: RFE  -- Highest RFE comes to 54% still lesser than chi2

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import warnings
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

def rfeFeature(indep_X, dep_Y, n ):
    rfeList=[]
    
    log_model = LogisticRegression(solver='lbfgs')
    RF = RandomForestClassifier(n_estimators = 10, criterion='entropy', random_state=0)
    DT = DecisionTreeClassifier(criterion='gini', max_features='sqrt', splitter='best', random_state=0)
    svc_model = SVC(kernel='linear', random_state=0)
    NB = GaussianNB()
    knn = KNeighborsClassifier(n_neighbors=5, metric = 'minkowski', p = 2)
    
    rfeModelList = [log_model, RF, svc_model, DT]
    
    for i in rfeModelList:
        print(i)
        log_rfe = RFE(i, n)#test = SelectKBest(score_func=chi2, k= n)
        fit1 = log_rfe.fit(indep_X, dep_Y)
        selected_columns = indep_X.columns[log_rfe.get_support()]  # Retrieve selected feature names
        print(f"Selected features for model {i}: {list(selected_columns)}")  # Print the selected feature names

        
        log_rfe_feature = fit1.transform(indep_X)
        
        rfeList.append(log_rfe_feature)
    return rfeList
def rfe_Classification_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    RFE_dataframe = pd.DataFrame(index=['Logistic', 'SVC', 'Random', 'DecisionTree' ], columns = ['Logistic', 'SVMl','SVMnl', 'KNN', 'Navie', 'Decision', 'Random' ])
    print(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    for number, idex in enumerate(RFE_dataframe.index):
        RFE_dataframe['Logistic'][idex]=acclog[number]
        RFE_dataframe['SVMl'][idex]=accsvml[number]
        RFE_dataframe['SVMnl'][idex]=accsvmnl[number]
        RFE_dataframe['KNN'][idex]=accknn[number]
        RFE_dataframe['Navie'][idex]=accnav[number]
        RFE_dataframe['Decision'][idex]=accdes[number]
        RFE_dataframe['Random'][idex]=accrf[number]
    return RFE_dataframe


#for j in [3]:  
rfeList = rfeFeature(indep_X, dep_Y, 3)
acclog = [] 
accsvml= [] 
accsvmnl= []
accknn= [] 
accnav= [] 
accdes= []
accrf = []
for i in rfeList:
    X_train, X_test, y_train, y_test = split_scalar (i, dep_Y)
    classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train, y_train, X_test)
    acclog.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train, y_train, X_test)
    accsvml.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = svm_NL(X_train, y_train, X_test)
    accsvmnl.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = knn(X_train, y_train, X_test)
    accknn.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train, y_train, X_test)
    accnav.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train, y_train, X_test)
    accdes.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = random(X_train, y_train, X_test)
    accrf.append(Accuracy)

result = rfe_Classification_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
result 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Selected features for model LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False): ['Server Status_s6', 'Server Status_s7', 'Data_Progress_inprogress']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
               

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
Logistic,0.531847,0.531847,0.531847,0.522293,0.531847,0.531847,0.531847
SVC,0.496815,0.496815,0.471338,0.493631,0.484076,0.496815,0.544586
Random,0.531847,0.531847,0.531847,0.522293,0.531847,0.531847,0.531847
DecisionTree,0.496815,0.496815,0.471338,0.493631,0.484076,0.496815,0.544586


In [111]:
# 4 A 2) Feature Selection: RFE  -- 

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import warnings
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

def rfeFeature(indep_X, dep_Y, n ):
    rfeList=[]
    
    log_model = LogisticRegression(solver='lbfgs')
    RF = RandomForestClassifier(n_estimators = 10, criterion='entropy', random_state=0)
    DT = DecisionTreeClassifier(criterion='gini', max_features='sqrt', splitter='best', random_state=0)
    svc_model = SVC(kernel='linear', random_state=0)
    NB = GaussianNB()
    knn = KNeighborsClassifier(n_neighbors=5, metric = 'minkowski', p = 2)
    
    rfeModelList = [log_model, RF, svc_model, DT]
    
    for i in rfeModelList:
        print(i)
        log_rfe = RFE(i, n)#test = SelectKBest(score_func=chi2, k= n)
        fit1 = log_rfe.fit(indep_X, dep_Y)
        selected_columns = indep_X.columns[log_rfe.get_support()]  # Retrieve selected feature names
        print(f"Selected features for model {i}: {list(selected_columns)}")  # Print the selected feature names

        
        log_rfe_feature = fit1.transform(indep_X)
        
        rfeList.append(log_rfe_feature)
    return rfeList
def rfe_Classification_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    RFE_dataframe = pd.DataFrame(index=['Logistic', 'SVC', 'Random', 'DecisionTree' ], columns = ['Logistic', 'SVMl','SVMnl', 'KNN', 'Navie', 'Decision', 'Random' ])
    print(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    for number, idex in enumerate(RFE_dataframe.index):
        RFE_dataframe['Logistic'][idex]=acclog[number]
        RFE_dataframe['SVMl'][idex]=accsvml[number]
        RFE_dataframe['SVMnl'][idex]=accsvmnl[number]
        RFE_dataframe['KNN'][idex]=accknn[number]
        RFE_dataframe['Navie'][idex]=accnav[number]
        RFE_dataframe['Decision'][idex]=accdes[number]
        RFE_dataframe['Random'][idex]=accrf[number]
    return RFE_dataframe


#for j in [4]:  
rfeList = rfeFeature(indep_X, dep_Y, 4)
acclog = [] 
accsvml= [] 
accsvmnl= []
accknn= [] 
accnav= [] 
accdes= []
accrf = []
for i in rfeList:
    X_train, X_test, y_train, y_test = split_scalar (i, dep_Y)
    classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train, y_train, X_test)
    acclog.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train, y_train, X_test)
    accsvml.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = svm_NL(X_train, y_train, X_test)
    accsvmnl.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = knn(X_train, y_train, X_test)
    accknn.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train, y_train, X_test)
    accnav.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train, y_train, X_test)
    accdes.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = random(X_train, y_train, X_test)
    accrf.append(Accuracy)

result = rfe_Classification_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
result 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Selected features for model LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False): ['Server Status_s5', 'Server Status_s6', 'Server Status_s7', 'Data_Progress_inprogress']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=N

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
Logistic,0.531847,0.531847,0.531847,0.522293,0.531847,0.531847,0.531847
SVC,0.519108,0.496815,0.5,0.449045,0.493631,0.506369,0.496815
Random,0.531847,0.531847,0.531847,0.522293,0.531847,0.531847,0.531847
DecisionTree,0.519108,0.496815,0.5,0.449045,0.493631,0.506369,0.496815


In [112]:
# 4 A 2) Feature Selection: RFE  -- 

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import warnings
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

def rfeFeature(indep_X, dep_Y, n ):
    rfeList=[]
    
    log_model = LogisticRegression(solver='lbfgs')
    RF = RandomForestClassifier(n_estimators = 10, criterion='entropy', random_state=0)
    DT = DecisionTreeClassifier(criterion='gini', max_features='sqrt', splitter='best', random_state=0)
    svc_model = SVC(kernel='linear', random_state=0)
    NB = GaussianNB()
    knn = KNeighborsClassifier(n_neighbors=5, metric = 'minkowski', p = 2)
    
    rfeModelList = [log_model, RF, svc_model, DT]
    
    for i in rfeModelList:
        print(i)
        log_rfe = RFE(i, n)#test = SelectKBest(score_func=chi2, k= n)
        fit1 = log_rfe.fit(indep_X, dep_Y)
        selected_columns = indep_X.columns[log_rfe.get_support()]  # Retrieve selected feature names
        print(f"Selected features for model {i}: {list(selected_columns)}")  # Print the selected feature names

        
        log_rfe_feature = fit1.transform(indep_X)
        
        rfeList.append(log_rfe_feature)
    return rfeList
def rfe_Classification_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    RFE_dataframe = pd.DataFrame(index=['Logistic', 'SVC', 'Random', 'DecisionTree' ], columns = ['Logistic', 'SVMl','SVMnl', 'KNN', 'Navie', 'Decision', 'Random' ])
    print(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    for number, idex in enumerate(RFE_dataframe.index):
        RFE_dataframe['Logistic'][idex]=acclog[number]
        RFE_dataframe['SVMl'][idex]=accsvml[number]
        RFE_dataframe['SVMnl'][idex]=accsvmnl[number]
        RFE_dataframe['KNN'][idex]=accknn[number]
        RFE_dataframe['Navie'][idex]=accnav[number]
        RFE_dataframe['Decision'][idex]=accdes[number]
        RFE_dataframe['Random'][idex]=accrf[number]
    return RFE_dataframe


#for j in [7]:  
rfeList = rfeFeature(indep_X, dep_Y, 7)
acclog = [] 
accsvml= [] 
accsvmnl= []
accknn= [] 
accnav= [] 
accdes= []
accrf = []
for i in rfeList:
    X_train, X_test, y_train, y_test = split_scalar (i, dep_Y)
    classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train, y_train, X_test)
    acclog.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train, y_train, X_test)
    accsvml.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = svm_NL(X_train, y_train, X_test)
    accsvmnl.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = knn(X_train, y_train, X_test)
    accknn.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train, y_train, X_test)
    accnav.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train, y_train, X_test)
    accdes.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = random(X_train, y_train, X_test)
    accrf.append(Accuracy)

result = rfe_Classification_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
result 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Selected features for model LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False): ['Server Status_s3', 'Server Status_s4', 'Server Status_s5', 'Server Status_s6', 'Server Status_s7', 'Data_Progress_inprogress', 'Server_Load_low']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
           

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
Logistic,0.547771,0.531847,0.531847,0.512739,0.544586,0.506369,0.509554
SVC,0.503185,0.531847,0.535032,0.547771,0.474522,0.512739,0.5
Random,0.547771,0.531847,0.531847,0.512739,0.544586,0.506369,0.509554
DecisionTree,0.484076,0.484076,0.496815,0.477707,0.487261,0.509554,0.471338


In [113]:
# 4 A 2) Feature Selection: RFE  -- 

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import warnings
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

def rfeFeature(indep_X, dep_Y, n ):
    rfeList=[]
    
    log_model = LogisticRegression(solver='lbfgs')
    RF = RandomForestClassifier(n_estimators = 10, criterion='entropy', random_state=0)
    DT = DecisionTreeClassifier(criterion='gini', max_features='sqrt', splitter='best', random_state=0)
    svc_model = SVC(kernel='linear', random_state=0)
    NB = GaussianNB()
    knn = KNeighborsClassifier(n_neighbors=5, metric = 'minkowski', p = 2)
    
    rfeModelList = [log_model, RF, svc_model, DT]
    
    for i in rfeModelList:
        print(i)
        log_rfe = RFE(i, n)#test = SelectKBest(score_func=chi2, k= n)
        fit1 = log_rfe.fit(indep_X, dep_Y)
        selected_columns = indep_X.columns[log_rfe.get_support()]  # Retrieve selected feature names
        print(f"Selected features for model {i}: {list(selected_columns)}")  # Print the selected feature names

        
        log_rfe_feature = fit1.transform(indep_X)
        
        rfeList.append(log_rfe_feature)
    return rfeList
def rfe_Classification_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    RFE_dataframe = pd.DataFrame(index=['Logistic', 'SVC', 'Random', 'DecisionTree' ], columns = ['Logistic', 'SVMl','SVMnl', 'KNN', 'Navie', 'Decision', 'Random' ])
    print(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    for number, idex in enumerate(RFE_dataframe.index):
        RFE_dataframe['Logistic'][idex]=acclog[number]
        RFE_dataframe['SVMl'][idex]=accsvml[number]
        RFE_dataframe['SVMnl'][idex]=accsvmnl[number]
        RFE_dataframe['KNN'][idex]=accknn[number]
        RFE_dataframe['Navie'][idex]=accnav[number]
        RFE_dataframe['Decision'][idex]=accdes[number]
        RFE_dataframe['Random'][idex]=accrf[number]
    return RFE_dataframe


#for j in [7]:  
rfeList = rfeFeature(indep_X, dep_Y, 13)
acclog = [] 
accsvml= [] 
accsvmnl= []
accknn= [] 
accnav= [] 
accdes= []
accrf = []
for i in rfeList:
    X_train, X_test, y_train, y_test = split_scalar (i, dep_Y)
    classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train, y_train, X_test)
    acclog.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train, y_train, X_test)
    accsvml.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = svm_NL(X_train, y_train, X_test)
    accsvmnl.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = knn(X_train, y_train, X_test)
    accknn.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train, y_train, X_test)
    accnav.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train, y_train, X_test)
    accdes.append(Accuracy)
    classifier, Accuracy, report, X_test, y_test, cm = random(X_train, y_train, X_test)
    accrf.append(Accuracy)

result = rfe_Classification_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
result 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Selected features for model LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False): ['Network Traffic (MB/s)', 'Request Size (MB)', 'Threshold', 'Response Time (ms)', 'Server Status_s2', 'Server Status_s3', 'Server Status_s4', 'Server Status_s5', 'Server Status_s6', 'Server Status_s7', 'Data_Progress_inprogress', 'Server_Load_low', 'Server_Load_medium']
RandomForestClassifier(bootstrap=True, class_we

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
Logistic,0.493631,0.5,0.522293,0.531847,0.528662,0.525478,0.493631
SVC,0.493631,0.5,0.522293,0.531847,0.528662,0.525478,0.493631
Random,0.493631,0.5,0.522293,0.531847,0.528662,0.525478,0.493631
DecisionTree,0.493631,0.5,0.522293,0.531847,0.528662,0.525478,0.493631


In [115]:
# 4 A 3) Feature Selection: PCA  -- Highest PCA comes to 51% still lesser than chi2
#PCA n : 3 
from sklearn.decomposition import PCA
def principleComponentAnalysis(X_train, X_test, n ):
    pca = PCA(n_components = n)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
     
    explained_variance = pca.explained_variance_
      
    return X_train_pca, X_test_pca, explained_variance

def PCA_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    PCA_dataframe = pd.DataFrame(index=["3" ], columns = ['Logistic', 'SVMl','SVMnl', 'KNN', 'Navie', 'Decision', 'Random' ])
    print(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    for number, idex in enumerate(PCA_dataframe.index):
        PCA_dataframe['Logistic'][idex]=acclog[number]
        PCA_dataframe['SVMl'][idex]=accsvml[number]
        PCA_dataframe['SVMnl'][idex]=accsvmnl[number]
        PCA_dataframe['KNN'][idex]=accknn[number]
        PCA_dataframe['Navie'][idex]=accnav[number]
        PCA_dataframe['Decision'][idex]=accdes[number]
        PCA_dataframe['Random'][idex]=accrf[number]
    return PCA_dataframe


X_train, X_test, y_train, y_test = split_scalar (indep_X, dep_Y)
X_train_pca, X_test_pca, explained_variance = principleComponentAnalysis(X_train, X_test, 3)
 
acclog = [] 
accsvml= [] 
accsvmnl= []
accknn= [] 
accnav= [] 
accdes= []
accrf = []

classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train_pca, y_train, X_test_pca)
acclog.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train_pca, y_train, X_test_pca)
accsvml.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_NL(X_train_pca, y_train, X_test_pca)
accsvmnl.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = knn(X_train_pca, y_train, X_test_pca)
accknn.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train_pca, y_train, X_test_pca)
accnav.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train_pca, y_train, X_test_pca)
accdes.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = random(X_train_pca, y_train, X_test_pca)
accrf.append(Accuracy)

result = PCA_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
result 

1 1 1 1 1 1 1


,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
3,0.490446,0.484076,0.490446,0.519108,0.464968,0.531847,0.519108


In [116]:
# 4 A 3) Feature Selection: PCA n : 4 
from sklearn.decomposition import PCA
def principleComponentAnalysis(X_train, X_test, n ):
    pca = PCA(n_components = n)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
     
    explained_variance = pca.explained_variance_
      
    return X_train_pca, X_test_pca, explained_variance

def PCA_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    PCA_dataframe = pd.DataFrame(index=["3" ], columns = ['Logistic', 'SVMl','SVMnl', 'KNN', 'Navie', 'Decision', 'Random' ])
    print(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    for number, idex in enumerate(PCA_dataframe.index):
        PCA_dataframe['Logistic'][idex]=acclog[number]
        PCA_dataframe['SVMl'][idex]=accsvml[number]
        PCA_dataframe['SVMnl'][idex]=accsvmnl[number]
        PCA_dataframe['KNN'][idex]=accknn[number]
        PCA_dataframe['Navie'][idex]=accnav[number]
        PCA_dataframe['Decision'][idex]=accdes[number]
        PCA_dataframe['Random'][idex]=accrf[number]
    return PCA_dataframe


X_train, X_test, y_train, y_test = split_scalar (indep_X, dep_Y)
X_train_pca, X_test_pca, explained_variance = principleComponentAnalysis(X_train, X_test, 4)
 
acclog = [] 
accsvml= [] 
accsvmnl= []
accknn= [] 
accnav= [] 
accdes= []
accrf = []

classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train_pca, y_train, X_test_pca)
acclog.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train_pca, y_train, X_test_pca)
accsvml.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_NL(X_train_pca, y_train, X_test_pca)
accsvmnl.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = knn(X_train_pca, y_train, X_test_pca)
accknn.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train_pca, y_train, X_test_pca)
accnav.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train_pca, y_train, X_test_pca)
accdes.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = random(X_train_pca, y_train, X_test_pca)
accrf.append(Accuracy)

result = PCA_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
result 

1 1 1 1 1 1 1


,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
3,0.471338,0.474522,0.509554,0.509554,0.487261,0.496815,0.509554


In [117]:
# 4 A 3) Feature Selection: PCA n : 7
from sklearn.decomposition import PCA
def principleComponentAnalysis(X_train, X_test, n ):
    pca = PCA(n_components = n)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
     
    explained_variance = pca.explained_variance_
      
    return X_train_pca, X_test_pca, explained_variance

def PCA_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    PCA_dataframe = pd.DataFrame(index=["3" ], columns = ['Logistic', 'SVMl','SVMnl', 'KNN', 'Navie', 'Decision', 'Random' ])
    print(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    for number, idex in enumerate(PCA_dataframe.index):
        PCA_dataframe['Logistic'][idex]=acclog[number]
        PCA_dataframe['SVMl'][idex]=accsvml[number]
        PCA_dataframe['SVMnl'][idex]=accsvmnl[number]
        PCA_dataframe['KNN'][idex]=accknn[number]
        PCA_dataframe['Navie'][idex]=accnav[number]
        PCA_dataframe['Decision'][idex]=accdes[number]
        PCA_dataframe['Random'][idex]=accrf[number]
    return PCA_dataframe


X_train, X_test, y_train, y_test = split_scalar (indep_X, dep_Y)
X_train_pca, X_test_pca, explained_variance = principleComponentAnalysis(X_train, X_test, 7)
 
acclog = [] 
accsvml= [] 
accsvmnl= []
accknn= [] 
accnav= [] 
accdes= []
accrf = []

classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train_pca, y_train, X_test_pca)
acclog.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train_pca, y_train, X_test_pca)
accsvml.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_NL(X_train_pca, y_train, X_test_pca)
accsvmnl.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = knn(X_train_pca, y_train, X_test_pca)
accknn.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train_pca, y_train, X_test_pca)
accnav.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train_pca, y_train, X_test_pca)
accdes.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = random(X_train_pca, y_train, X_test_pca)
accrf.append(Accuracy)

result = PCA_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
result 

1 1 1 1 1 1 1


,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
3,0.484076,0.490446,0.506369,0.525478,0.519108,0.477707,0.509554


In [118]:
# 4 A 3) Feature Selection: PCA n : 10
from sklearn.decomposition import PCA
def principleComponentAnalysis(X_train, X_test, n ):
    pca = PCA(n_components = n)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
     
    explained_variance = pca.explained_variance_
      
    return X_train_pca, X_test_pca, explained_variance

def PCA_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    PCA_dataframe = pd.DataFrame(index=["3" ], columns = ['Logistic', 'SVMl','SVMnl', 'KNN', 'Navie', 'Decision', 'Random' ])
    print(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    for number, idex in enumerate(PCA_dataframe.index):
        PCA_dataframe['Logistic'][idex]=acclog[number]
        PCA_dataframe['SVMl'][idex]=accsvml[number]
        PCA_dataframe['SVMnl'][idex]=accsvmnl[number]
        PCA_dataframe['KNN'][idex]=accknn[number]
        PCA_dataframe['Navie'][idex]=accnav[number]
        PCA_dataframe['Decision'][idex]=accdes[number]
        PCA_dataframe['Random'][idex]=accrf[number]
    return PCA_dataframe


X_train, X_test, y_train, y_test = split_scalar (indep_X, dep_Y)
X_train_pca, X_test_pca, explained_variance = principleComponentAnalysis(X_train, X_test, 10)
 
acclog = [] 
accsvml= [] 
accsvmnl= []
accknn= [] 
accnav= [] 
accdes= []
accrf = []

classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train_pca, y_train, X_test_pca)
acclog.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train_pca, y_train, X_test_pca)
accsvml.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_NL(X_train_pca, y_train, X_test_pca)
accsvmnl.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = knn(X_train_pca, y_train, X_test_pca)
accknn.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train_pca, y_train, X_test_pca)
accnav.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train_pca, y_train, X_test_pca)
accdes.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = random(X_train_pca, y_train, X_test_pca)
accrf.append(Accuracy)

result = PCA_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
result 

1 1 1 1 1 1 1


,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
3,0.490446,0.503185,0.519108,0.519108,0.512739,0.515924,0.490446


In [127]:
# 4 A 4) Feature Selection: LDA  -- Highest LDA comes to 53% still lesser than chi2

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
def linearDiscriminantAnalysis(X_train,y_train, X_test, n ):
    lda = LinearDiscriminantAnalysis(n_components = n)
    X_train_lda = lda.fit_transform(X_train, y_train)
    X_test_lda = lda.transform(X_test)
      
      
    return X_train_lda, X_test_lda 
def LDA_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    PCA_dataframe = pd.DataFrame(index=["5" ], columns = ['Logistic', 'SVMl','SVMnl', 'KNN', 'Navie', 'Decision', 'Random' ])
    print(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    for number, idex in enumerate(PCA_dataframe.index):
        PCA_dataframe['Logistic'][idex]=acclog[number]
        PCA_dataframe['SVMl'][idex]=accsvml[number]
        PCA_dataframe['SVMnl'][idex]=accsvmnl[number]
        PCA_dataframe['KNN'][idex]=accknn[number]
        PCA_dataframe['Navie'][idex]=accnav[number]
        PCA_dataframe['Decision'][idex]=accdes[number]
        PCA_dataframe['Random'][idex]=accrf[number]
    return PCA_dataframe



X_train, X_test, y_train, y_test = split_scalar (indep_X, dep_Y)
X_train_lda, X_test_lda  = linearDiscriminantAnalysis(X_train,y_train, X_test, 5)
 
acclog = [] 
accsvml= [] 
accsvmnl= []
accknn= [] 
accnav= [] 
accdes= []
accrf = []

classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train_lda, y_train, X_test_lda)
acclog.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train_lda, y_train, X_test_lda)
accsvml.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_NL(X_train_lda, y_train, X_test_lda)
accsvmnl.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = knn(X_train_lda, y_train, X_test_lda)
accknn.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train_lda, y_train, X_test_lda)
accnav.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train_lda, y_train, X_test_lda)
accdes.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = random(X_train_lda, y_train, X_test_lda)
accrf.append(Accuracy)

result = LDA_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
result 


1 1 1 1 1 1 1


,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
5,0.493631,0.493631,0.531847,0.474522,0.509554,0.461783,0.477707


In [141]:
# 4 A 5) Feature Selection: KernalPCA  -- Highest KernalPCA comes to 50% still lesser than chi2

from sklearn.decomposition import KernelPCA
def kernalPCA(X_train, X_test, n ):
    kernalpca = KernelPCA(n_components = n, kernel='rbf')
    X_train = kernalpca.fit_transform(X_train )
    X_test = kernalpca.transform(X_test)
      
      
    return X_train, X_test
def LDA_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    PCA_dataframe = pd.DataFrame(index=["4" ], columns = ['Logistic', 'SVMl','SVMnl', 'KNN', 'Navie', 'Decision', 'Random' ])
    print(len(acclog), len(accsvml), len(accsvmnl), len(accknn), len(accnav), len(accdes), len(accrf))
    for number, idex in enumerate(PCA_dataframe.index):
        PCA_dataframe['Logistic'][idex]=acclog[number]
        PCA_dataframe['SVMl'][idex]=accsvml[number]
        PCA_dataframe['SVMnl'][idex]=accsvmnl[number]
        PCA_dataframe['KNN'][idex]=accknn[number]
        PCA_dataframe['Navie'][idex]=accnav[number]
        PCA_dataframe['Decision'][idex]=accdes[number]
        PCA_dataframe['Random'][idex]=accrf[number]
    return PCA_dataframe



X_train, X_test, y_train, y_test = split_scalar (indep_X, dep_Y)
X_train_lda, X_test_lda  = kernalPCA(X_train, X_test, 4)
 
acclog = [] 
accsvml= [] 
accsvmnl= []
accknn= [] 
accnav= [] 
accdes= []
accrf = []

classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train_lda, y_train, X_test_lda)
acclog.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train_lda, y_train, X_test_lda)
accsvml.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = svm_NL(X_train_lda, y_train, X_test_lda)
accsvmnl.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = knn(X_train_lda, y_train, X_test_lda)
accknn.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train_lda, y_train, X_test_lda)
accnav.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train_lda, y_train, X_test_lda)
accdes.append(Accuracy)
classifier, Accuracy, report, X_test, y_test, cm = random(X_train_lda, y_train, X_test_lda)
accrf.append(Accuracy)

result = LDA_create_Table(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
result 


1 1 1 1 1 1 1


,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
4,0.522293,0.503185,0.509554,0.480892,0.474522,0.506369,0.471338


In [144]:
# 4 A 5) Feature Selection: Grid SVM
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

param_grid = {'kernel': [ 'poly'], #'linear', 'poly', 'sigmoid'],
              'C': [1], #1], #, 10, 100, 1000, 3000],
              'gamma': [1], #scale'], #, 'auto'],
              'degree': [2],
              'epsilon' :  [0.001]
             }

grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 3, n_jobs = 2)

#create the model for all the Grid combinations
grid.fit(X_train, y_train)

print("Printing best param after tuning: ", grid.best_params_, " best score: ", grid.best_score_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [146]:

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

param_grid = { 'criterion': [ "absolute_error", "mae" ], #"mse", "mae", "friedman_mse", "absolute_error", "poisson"], 
              'splitter' : ["best", "random"],
              'max_features' : [None, "auto", "sqrt", "log2"]
             }

grid = GridSearchCV(DecisionTreeRegressor(), param_grid, refit = True, verbose = 3, n_jobs = 2)

#create the model for all the Grid combinations
grid.fit(X_train, y_train)

print("Printing before tuning: ", grid.best_params_, " best score: ", grid.best_score_)

y_grid_predictions = grid.predict(X_test)

from sklearn.metrics import r2_score
r_score = r2_score(y_test, y_grid_predictions)

print("The R score value for best param {}: ".format(grid.best_params_), r_score)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.
